In [1]:
import os
import pandas as pd
import numpy as np

folder_path = '../ssd_data/smartlog2019ssd'
file_extension = '.csv'

for month in range(3, 4):
    target_month = f'2019{str(month).zfill(2)}'
    dfs = []  # dfs 리스트 초기화
    for file_name in os.listdir(folder_path):
        if file_name.endswith(file_extension):
            date = file_name
            date = date.split('.')[0]  # 파일명에서 날짜 추출
            file_month = date[:6]  # 날짜에서 월 추출
            if target_month == file_month:
                file_path = os.path.join(folder_path, file_name)
                df = pd.read_csv(file_path)
                dfs.append(df)

    combined_df = pd.concat(dfs, ignore_index=True)
    combined_df_name = f'combined_df{str(month).zfill(2)}'
    globals()[combined_df_name] = combined_df

In [2]:
combined_df03.shape

(12658448, 105)

In [3]:
# null값이 있는 열을 제거하는 코드
def remove_null_columns(df):
    # null인 컬럼을 확인하고, 모든 값이 null인 컬럼을 찾습니다.
    null_columns = df.columns[df.isnull().all()]
    
    # 모든 값이 null인 컬럼을 DataFrame에서 제거합니다.
    df = df.drop(null_columns, axis=1)
    
    return df

In [4]:
# 하나의 unique한 값으로만 이루어진 컬럼을 제거하는 코드
def remove_single_value_columns(df):
    single_value_cols = []
    for col in df.columns:
        unique_values = df[col].nunique()
        if unique_values == 1:
            single_value_cols.append(col)

    df_filtered = df.drop(single_value_cols, axis=1)
    return df_filtered


In [5]:
combined_df03 = remove_null_columns(combined_df03)

In [6]:
combined_df03.shape

(12658448, 69)

In [7]:
combined_df03= remove_single_value_columns(combined_df03)

In [8]:
combined_df03.shape

(12658448, 58)

In [9]:
print(combined_df03.columns)

Index(['disk_id', 'ds', 'model', 'n_1', 'r_1', 'n_5', 'r_5', 'n_9', 'r_9',
       'n_12', 'r_12', 'n_170', 'r_170', 'n_171', 'r_171', 'n_172', 'r_172',
       'n_173', 'r_173', 'r_174', 'n_177', 'r_177', 'n_180', 'r_180', 'n_181',
       'r_181', 'n_182', 'r_182', 'n_183', 'r_183', 'n_184', 'r_184', 'n_187',
       'r_187', 'r_188', 'n_190', 'r_190', 'r_192', 'n_194', 'r_194', 'n_195',
       'r_195', 'n_196', 'r_196', 'r_197', 'r_198', 'n_199', 'r_199', 'r_206',
       'n_233', 'n_241', 'r_241', 'n_242', 'r_242', 'r_244', 'n_175', 'r_175',
       'n_232'],
      dtype='object')


In [10]:
combined_df03.iloc[:100]

,disk_id,ds,model,n_1,r_1,n_5,r_5,n_9,r_9,n_12,...,r_206,n_233,n_241,r_241,n_242,r_242,r_244,n_175,r_175,n_232
0,100005,20190301,MA2,NaN,NaN,100.0,0.0,100.0,31192.0,100.0,...,NaN,90.0,100.0,3193536.0,100.0,42611.0,NaN,100.0,7.991166e+11,100.0
1,100024,20190301,MC1,100.0,0.0,100.0,2.0,100.0,14736.0,100.0,...,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,100031,20190301,MC1,100.0,0.0,100.0,1.0,100.0,12208.0,100.0,...,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,10005,20190301,MC1,100.0,0.0,100.0,0.0,100.0,4084.0,100.0,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,100057,20190301,MA2,NaN,NaN,100.0,2.0,100.0,40465.0,100.0,...,NaN,89.0,100.0,6274404.0,100.0,278873.0,NaN,100.0,1.035318e+12,99.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,100818,20190301,MA2,NaN,NaN,100.0,0.0,100.0,38846.0,100.0,...,NaN,91.0,100.0,6528697.0,100.0,64793.0,NaN,100.0,9.922344e+11,100.0
96,100847,20190301,MC1,100.0,0.0,100.0,0.0,100.0,12845.0,100.0,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
97,100861,20190301,MA2,NaN,NaN,100.0,0.0,100.0,40265.0,100.0,...,NaN,69.0,100.0,14876743.0,100.0,40978569.0,NaN,100.0,1.027058e+12,100.0
98,100867,20190301,MC1,100.0,0.0,100.0,0.0,100.0,5661.0,100.0,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
# combined_df04_sample = combined_df04.sample(n=1000000, random_state=42)

In [12]:
# combined_df04_sample.shape

In [13]:
# combined_df04_sample.to_csv('combined_df01_sample.csv', index=False)

In [14]:
#라벨링 하기 label data 불러오기
label = pd.read_csv('../ssd_data/ssd_failure_label.csv')
label

,model,failure_time,disk_id
0,MA2,2019-03-22 10:24:38,4711
1,MA2,2019-02-22 04:56:06,82064
2,MA2,2018-12-17 12:16:33,32311
3,MA2,2018-05-19 17:32:03,18316
4,MA2,2018-10-25 04:00:50,32466
...,...,...,...
16300,MC2,2019-09-30 09:09:55,15910
16301,MC2,2019-05-30 10:52:18,837
16302,MC2,2019-01-19 05:35:06,13236
16303,MC2,2019-07-18 23:14:27,277


In [15]:
#label data 전처리
label['failure_time']=label['failure_time'].str[:10].str.replace('-', '')
label['failure_time']=label['failure_time'].astype('int64')
label['failure_time'].dtypes

dtype('int64')

In [16]:
# 조인 수행
combined_df03 = combined_df03.merge(label, how='left', left_on=['model', 'disk_id', 'ds'],right_on=['model', 'disk_id', 'failure_time'])

# label 열 생성 및 라벨링 수행
combined_df03['label'] = np.where(combined_df03['failure_time'].notnull(), 1, 0)
del(combined_df03['failure_time'])

In [17]:
combined_df03.shape

(12658448, 59)

In [18]:
combined_df03[combined_df03['label']==1]

,disk_id,ds,model,n_1,r_1,n_5,r_5,n_9,r_9,n_12,...,n_233,n_241,r_241,n_242,r_242,r_244,n_175,r_175,n_232,label
146484,8197,20190301,MB1,NaN,NaN,99.0,2.0,97.0,14121.0,99.0,...,NaN,99.0,9.063744e+10,99.0,3.816379e+10,0.0,NaN,NaN,NaN,1
214242,32187,20190301,MA1,130.0,4.294967e+09,98.0,4.0,100.0,38667.0,100.0,...,NaN,NaN,NaN,NaN,NaN,NaN,100.0,9.751467e+11,NaN,1
307139,33133,20190301,MB2,NaN,NaN,100.0,0.0,99.0,66.0,99.0,...,NaN,99.0,3.139740e+05,99.0,1.201100e+04,0.0,NaN,NaN,NaN,1
321957,187914,20190301,MC1,100.0,1.100000e+01,100.0,93.0,100.0,11019.0,100.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
485469,12983,20190302,MA1,130.0,4.294967e+09,94.0,0.0,100.0,48160.0,100.0,...,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.224183e+12,NaN,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12199331,9758,20190330,MB1,NaN,NaN,99.0,2.0,96.0,18127.0,99.0,...,NaN,99.0,8.366342e+11,99.0,6.027516e+11,0.0,NaN,NaN,NaN,1
12266795,40330,20190331,MA1,130.0,9.576000e+03,97.0,0.0,100.0,41285.0,100.0,...,NaN,NaN,NaN,NaN,NaN,NaN,100.0,1.039715e+12,NaN,1
12485165,31909,20190331,MA1,130.0,4.294967e+09,93.0,0.0,100.0,48830.0,100.0,...,NaN,NaN,NaN,NaN,NaN,NaN,100.0,1.228697e+12,NaN,1
12579648,4848,20190331,MA1,130.0,3.048000e+03,98.0,0.0,100.0,38618.0,100.0,...,NaN,NaN,NaN,NaN,NaN,NaN,100.0,9.752904e+11,NaN,1


In [19]:
combined_df03.to_csv('combined_df03.csv', index=False)

In [34]:
data0404 = pd.DataFrame([])
for chunk in pd.read_csv("combined_df04.csv", chunksize=1000000):
    data0404 = pd.concat([data0404, chunk], ignore_index=True)
data0404

,disk_id,ds,model,n_1,r_1,n_5,r_5,n_9,r_9,n_12,...,n_233,n_241,r_241,n_242,r_242,r_244,n_175,r_175,n_232,label
0,100005,20190401,MC1,100.0,0.0,100.0,0.0,100.0,14398.0,100.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
1,100006,20190401,MA2,NaN,NaN,100.0,0.0,100.0,28461.0,100.0,...,97.0,100.0,757909.0,100.0,13378.0,NaN,100.0,7.262268e+11,100.0,0
2,100021,20190401,MC1,100.0,2.0,100.0,1.0,100.0,17845.0,100.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
3,10003,20190401,MC1,100.0,0.0,100.0,0.0,100.0,4827.0,100.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
4,100074,20190401,MC1,100.0,0.0,100.0,0.0,100.0,10697.0,100.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11904110,99930,20190430,MA2,NaN,NaN,100.0,0.0,100.0,31639.0,100.0,...,88.0,100.0,4238858.0,100.0,59206.0,NaN,100.0,8.080853e+11,100.0,0
11904111,99931,20190430,MA2,NaN,NaN,100.0,0.0,100.0,33359.0,100.0,...,70.0,100.0,9059332.0,100.0,255843.0,NaN,100.0,8.463770e+11,100.0,0
11904112,9997,20190430,MA2,NaN,NaN,100.0,0.0,100.0,31684.0,100.0,...,99.0,100.0,563602.0,100.0,46004.0,NaN,100.0,8.118662e+11,100.0,0
11904113,99993,20190430,MC1,100.0,0.0,100.0,1.0,100.0,7019.0,100.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
